# Demo For "First Order Motion Model for Image Animation"

1. Import specific dependencies
2. Mount drive (must have Google account)

In [0]:
!git clone https://github.com/AliaksandrSiarohin/first-order-model
cd first-order-model

from google.colab import drive
drive.mount('/content/gdrive')

Cloning into 'first-order-model'...
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 219 (delta 0), reused 1 (delta 0), pack-reused 215
Receiving objects: 100% (219/219), 72.34 MiB | 28.83 MiB/s, done.
Resolving deltas: 100% (106/106), done.


**Add folder https://drive.google.com/drive/folders/1kZ1gCnpfU0BnpdU47pLM_TQ6RypDDqgw?usp=sharing  to your google drive.**

3. Prepare side-by-side-by-side video display
4. Load trained checkpoints from first-order-motion-model

In [0]:
import imageio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML
import warnings
warnings.filterwarnings("ignore")

from demo import load_checkpoints
from demo import make_animation
from skimage import img_as_ubyte

def display(source, driving, generated=None):
    fig = plt.figure(figsize=(8 + 4 * (generated is not None), 6))

    ims = []
    for i in range(len(driving)):
        cols = [source]
        cols.append(driving[i])
        if generated is not None:
            cols.append(generated[i])
        im = plt.imshow(np.concatenate(cols, axis=1), animated=True)
        plt.axis('off')
        ims.append([im])

    ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=1000)
    plt.close()
    return ani

generator, kp_detector = load_checkpoints(config_path='config/vox-256.yaml', 
                            checkpoint_path='/content/gdrive/My Drive/first-order-motion-model/vox-cpk.pth.tar')

5. Load reference image, driving video
6. Resize inputs
7. Generate predicted video (no sound)

In [0]:
#https://github.com/imageio/imageio/issues/168
def extract_frames(videofilename):
  try:
    frames = []
    for frame in imageio.get_reader(videofilename):
        frames.append(frame)
  except RuntimeError:
      print('runtime error')
  return frames

source_image = imageio.imread('/content/gdrive/My Drive/Colab Notebooks/CS485/final_project(not_repo)/watson_graphic.png')
driving_video = extract_frames('/content/gdrive/My Drive/Colab Notebooks/CS485/final_project(not_repo)/selfie_blah_out.mp4')


#Resize image and video to 256x256

source_image = resize(source_image, (256, 256))[..., :3]
driving_video = [resize(frame, (256, 256))[..., :3] for frame in driving_video]

predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=True, adapt_movement_scale=False)
imageio.mimsave('/content/gdrive/My Drive/Colab Notebooks/CS485/final_project(not_repo)/watson_default_video.mp4', [img_as_ubyte(frame) for frame in predictions])

HTML(display(source_image, driving_video, predictions).to_html5_video())

100%|██████████| 168/168 [00:06<00:00, 25.99it/s]
